In [ ]:
#!pip install textract
#!sudo apt-get install antiword

###Чтение документа

In [ ]:
import textract
import unicodedata
import chardet
import string

def extract_text(file_path):
    """
    Считывает текст по ссылке.

    Параметры:
    - file_path (str): ссылка на файл.

    Возвращает:
    - text (str): текст из документа.
    """
    file_extension = file_path.split('.')[-1].lower()
    if file_extension in ['docx','doc']:
        text = unicodedata.normalize("NFKD", textract.process(file_path).decode('utf-8'))
    elif file_extension == 'pdf':
        text = unicodedata.normalize("NFKD", textract.process(file_path).decode('utf-8'))
    else:
        text = "Формат файла не соответствует!"
    return text

###Структура ГОСТ

In [ ]:
import re

def gost(text, number):
  """
  Проверяет соответствие структуре ГОСТ.

  Параметры:
  - text (str): текст из документа,
  - number (int): тип документа.

  Возвращает:
  - result_gost (list): список списков, содержащиц в себе элемент структуры ГОСТ и его наличие/отсутствие в документе.
  """
  Tz_note = ["общие сведения", "цели и назначение создания автоматизированной системы", "характеристика объектов автоматизации",
             "требования к автоматизированной системе", "состав и содержание работ по созданию автоматизированной системы",
             "порядок разработки автоматизированной системы", "порядок контроля и приемки автоматизированной системы",
             "требования к составу и содержанию работ по подготовке объекта автоматизации к вводу автоматизированной системы в действие",
             "требования к документированию", "источники разработки"]
  Tz_note_check = ["общиесведения","целииназначениесозданияавтоматизированноисистемы|назначениеицели(?:развития)?(?:и)?(?:создания)?(?:подсистем)?системы",
                   "характеристикаобъектовавтоматизации|характеристикаобъектаавтоматизации", "требованияк(?:автоматизированнои)?системе",
                   "состависодержаниеработ(?:услуг)?(?:посозданиюавтоматизированнойсистемы|поразвитиюсистемы)|требованияксоставу(?:исодержанию)?работ(?:посозданиюсистемы|результатамсрокамиэтапамихвыполнения)",
                   "порядокразработкиавтоматизированноисистемы", "порядокконтроляиприемки(?:автоматизированнои)?(?:системы|работ)",
                   "требованияксоставуисодержаниюработ(?:услуг)?поподготовкеобъектаавтоматизацииквводу(?:автоматизированнои)?системывдеиствие", "требованиякдокументированию", "источникиразработки"]
  Explanatory_note = ["общие положения", "описание процесса деятельности", "основные технические решения", "мероприятия по подготовке объекта автоматизации к вводу системы в действие"]
  Explanatory_note_check = ["общиеположения", "описаниепроцессадеятельности", "основныетехническиерешения", "мероприятияпоподготовкеобъектаавтоматизации(?:\n)?квводусистемывдеиствие"]
  Report_note = ["список исполнителей", "реферат", "содержание", "термины и определения", "перечень сокращений и обозначений",
                 "введение", "заключение", "список использованных источников", "приложения"]
  Report_note_check = ["списокисполнителеи", "реферат", "содержание", "терминыиопределения", "переченьсокращениииобозначении",
                       "введение", "заключение", "списокиспользованныхисточников", "приложени(?:я|e)"]
  Report_note1 = ["аннотация", "объект опытной эксплуатации", "цель опытной эксплуатации",
                  "общие положения", "опытная эксплуатация", "порядок проведения опытной эксплуатации", "результаты проведения опытной эксплуатации",
                  "cписок сокращений и обозначений", "заключение"]
  Report_note_check1 = ["аннотация", "объектопытноиэксплуатации", "цельопытноиэксплуатации", "общиеположения", "опытнаяэксплуатация",
                        "порядокпроведенияопытноиэксплуатации|журналопытноиэксплуатации", "результатыпроведенияопытноиэксплуатации",
                        "списоксокращениииобозначении|определенияобозначенияисокращения", "заключение"]
  Check = [[Tz_note, Tz_note_check],[Explanatory_note, Explanatory_note_check],[Report_note, Report_note_check],[Report_note1, Report_note_check1]]
  cleaned_text = re.sub(r'[^а-яА-Я\n]', '', text.lower())
  cleaned_text = re.sub(r'\n+', '\n', cleaned_text)
  result_gost = []
  for i, j in zip(Check[number][1], Check[number][0]):
    if(len(re.findall('\n{1}'+str(i)+'{1}\n', cleaned_text)) >= 1):
        result_gost.append([j, True])
    else:
        result_gost.append([j, True])
  return result_gost

###Реализация антиплагиата

Очищение текста

In [ ]:
import string

def clean_text(text):
    """
    Очищает текст от элементов, которые не являются знакаму пунктуации, буквами или цифрами.

    Параметры:
    - text (str): текст из документа.

    Возвращает:
    - text (str): очищеннный текст документа.
    """
    text = re.sub(r'(\W)\1+', r'\1', text)
    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9{}_„“«»/\-‘’]+'.format(re.escape(string.punctuation)), ' ', text)
    return text

Запрос на проверку плагиата

In [ ]:
import requests
import json
import pandas as pd

def anti_plagiarism(userkey, text):
  """
  Возвразает uid запроса.

  Параметры:
  - userkey (str): ключ доступа для пользователя,
  - text (str): текст из документа.

  Возвращает в случае успеха:
  - response_data (dict): содержит uid запроса.
  Возвращает в случае неудачи:
  - (int): код ошибки.
  """
  # Данные для запроса
  body = {
      'userkey': userkey,
      'text': text,
      'callback': 'https://your-site.com/callback'}
  response = requests.post('https://api.text.ru/post', json=body)
  # Проверка успешности запроса
  if response.status_code != 200:
    raise Exception(f'Ошибка запроса: {response.status_code}')
    return response.status_code
  response_data = response.json()
  return response_data

In [ ]:
def anti_plagiarism_result(userkey, uid):
  """
  Возвразает информацию из антиплагиата.

  Параметры:
  - userkey (str): ключ доступа для пользователя,
  - uid (str): uid запроса.

  Возвращает:
  - data (dict): содержит информацию из антиплагиата.
  """
  body = {
      'userkey': userkey,
      'uid': uid}
  url = 'https://api.text.ru/post'
  response = requests.post(url, json=body)
  response.raise_for_status()
  data = response.json()
  return data

In [ ]:
def check_anti_plagiarism(userkey, text):
  """
  Возвразает обработанную информацию из антиплагиата.

  Параметры:
  - userkey (str): ключ доступа для пользователя,
  - text (str): текст документа.

  Возвращает в случае успеха:
  - (list): содержащий в себе информацию из антиплагиата, датафрейм найденных заимствований и значение уникальности текста.
  Возвращает в случае неудачи:
  - (str): информацию о произошедшей ошибке.
  """
  post = anti_plagiarism(userkey, text)
  df = pd.DataFrame()
  if "error_code" in post and "error_desc" in post:
    return f"Произошла ошибка {post['error_code']}: {post['error_desc']}", df
  else:
    answer = anti_plagiarism_result(userkey, post['text_uid'])
    while "error_code" in answer:
      answer = anti_plagiarism_result(userkey, post['text_uid'])
    text_unique = answer['text_unique']
    result_json = json.loads(answer['result_json'])
    res = []
    res.append(f"Дата окончания проверки текста на сервере: {result_json['date_check']}\n")
    res.append(f"Уникальность текста: {text_unique}%\n")
    res_str = ""
    if len(result_json['urls']) != 0:
      urls_pd = []
      pro_pd = []
      res.append(f"Список URL и процент совпадения текста:\n")
      for url_data in result_json['urls']:
        ur = url_data['url']
        res_str += f"URL: {ur}\n"
        urls_pd.append(ur)
        pr = url_data['plagiat']
        res_str += f"Процент совпадения текста: {pr}%\n"
        pro_pd.append(pr)
      if len(urls_pd) != 0:
        list_tuples = list(zip(urls_pd, pro_pd))
        df = pd.DataFrame(list_tuples, columns=['URL', '%'])
    res.append(res_str)
    return res, df, text_unique

###Выделение ключевых и частовстречающихся слов из текста

Часто встречающиеся слова

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from pymystem3 import Mystem
from nltk.probability import FreqDist

def remove_chars_from_text(text, chars):
  """
  Удаляет элементы из текста.

  Параметры:
  - text (str): текст документа,
  - chars (str): строка содержащая данные, которые нужно удалить.

  Возвращает:
  - (str): очищенную строку.
  """
  return "".join([ch for ch in text if ch not in chars])

def Text_processing(text):
  """
  Подсчитывает часто встречающиеся слова из текста.

  Параметры:
  - text (str): текст документа.

  Возвращает:
  - (class): счетчик распределения частот NLTK.
  """
  text = text.lower()
  mystem = Mystem()
  stops_word = ["менее", "более", "кроме", "должно", "должный", "данный", "детея"] + stopwords.words("russian") + stopwords.words("english")
  text = remove_chars_from_text(text, string.punctuation)  #удаление знаков пунктуации
  text = remove_chars_from_text(text, string.digits)  #удаление цифр
  tokenizer = RegexpTokenizer(r'\w+')
  text = tokenizer.tokenize(text)
  text = [mystem.lemmatize(word)[0] for word in text] #лемматизируем текст
  text = [i for i in text if i not in stops_word] #удаление стоп слов
  text = nltk.Text(text)
  return FreqDist(text)

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def word_cloud(data):
  """
  Рисует облако слов.

  Параметры:
  - data (object): датафрейм содержащий слово и число, которое равно количеству появления слова в документе.
  """
  # Создание текста для облака слов
  text = ' '.join(data)
  # Создание облака слов
  wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
  # Отображение облака слов
  plt.figure(figsize=(10, 5))
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis('off')
  # Сохранение облака слов в формате PNG
  plt.savefig('wordcloud.png', bbox_inches='tight', pad_inches=0)
  plt.show()

Метод RAKE

In [ ]:
#!pip install nlp-rake
#!pip install nltk

In [ ]:
from nlp_rake import Rake

def key_word_rake(text, stops_my, n, max_words):
  """
  Возвращает ключевые слова, используя метод RAKE.

  Параметры:
  - text (str): текст документа,
  - stops_my (list): стоп-слова,
  - n (int): количество возвращаемых слов,
  - max_words (int): максимальное числов слов в фразе.

  Возвращает:
  - (list): n найденных слов.
  """
  text = clean_text(text)
  text = re.sub(r'\s+\.', '.', text)
  text = remove_chars_from_text(text, string.digits)  #удаление цифр
  text = re.sub(r'\b\w{2}\b', '', text)
  rake = Rake(stopwords = stops_my, max_words = max_words)
  k = rake.apply(text)
  return k[:n]

Метод YAKE

In [ ]:
#!pip install yake

In [ ]:
import yake

def key_word_yake(text, n, top):
  """
  Возвращает ключевые слова, используя метод YAKE.

  Параметры:
  - text (str): текст документа,
  - n (int): максимальное количество слов в фразе,
  - top (int): количество ключевых слов.

  Возвращает:
  - (list): n найденных слов.
  """
  extractor = yake.KeywordExtractor(
      lan = "ru",
      n = n,
      dedupLim = 0.7,
      top = top
      )
  keywords = extractor.extract_keywords(text)
  filtered_keywords = [kw for kw in keywords if len(kw[0]) > 2]
  return sorted(filtered_keywords, key=lambda x: x[1], reverse=True)

TextRank

In [ ]:
#!pip install summa

In [ ]:
from summa import keywords
from pymystem3 import Mystem

def key_word_textrank(text, stops, n):
  """
  Возвращает ключевые слова, используя метод TextRank.

  Параметры:
  - text (str): текст документа,
  - stops (list): стоп слова,
  - n (int): количество ключевых слов.

  Возвращает:
  - (list): n найденных слов.
  """
  mystem = Mystem()
  text_clean = ""
  for word in text.split():
    lemma = mystem.lemmatize(word)[0]
    if lemma.lower() not in stops and len(lemma) > 2:
        text_clean += lemma + " "
  return keywords.keywords(text_clean, language = "russian").split("\n")[:n]

Topia

In [ ]:
#!pip install rutermextract

In [ ]:
from rutermextract import TermExtractor

def topia(cleaned_text, n):
  """
  Возвращает ключевые слова, используя метод Topia.

  Параметры:
  - cleaned_text (str): текст документа,
  - n (int): количество ключевых слов.

  Возвращает:
  - (list): содержащий в себе три элемента массив n ключевых слов, массив ключевых слов с их количеством и массив ключевых фраз с их количеством.
  """
  term_extractor = TermExtractor()
  single_word_terms = []
  key_word4 = []
  multi_word_terms = []
  for term in term_extractor(cleaned_text):
    if (re.search(r'\d', term.normalized) is not None) == False:
      if len(term.normalized.split()) == 1:
        single_word_terms.append((term.normalized, term.count))
        key_word4.append(term.normalized)
      else:
        multi_word_terms.append((term.normalized, term.count))
  return key_word4[:n], single_word_terms, multi_word_terms

Исправление слов используя YandexSpeller

In [ ]:
#!pip install pyaspeller

In [ ]:
from pyaspeller import YandexSpeller

def lemmatize_words(words):
  """
  Исправляет ошибки в написании слов.

  Параметры:
  - words (list): найденные слова.

  Возвращает:
  - (set): уникальные слова.
  """
  mystem = Mystem()
  speller = YandexSpeller()
  cleaned_words = []
  for i in words:
    new_i = re.sub(r'[^a-zA-Z0-9а-яА-Я]', '', i)
    if new_i != "":
      cleaned_words.append(new_i)
  lemmatized = mystem.lemmatize(" ".join(cleaned_words))
  return set(speller.spelled(word.strip()) for word in lemmatized if word.strip())

###РОСПАТЕНТ

In [ ]:
# Функция для распаковки имен из словарей
def unpack_names(names):
    """
    Преобразует данные в строку.

    Параметры:
    - names (list/dict/str): данные.

    Возвращает:
    - (set): преобразованные данные в виде строки.
    """
    if isinstance(names, list):
        return ', '.join(item['name'] for item in names)
    elif isinstance(names, dict):
        return names['name']
    else:
        return str(names)

# Функция для удаления <em> и </em> из строк
def remove_em_tags(value):
    """
    Удаляет элементы из строки.

    Параметры:
    - value (str): данные.

    Возвращает:
    - (str): очищенная строка.
    """
    if isinstance(value, str):
        return value.replace('<em>', '').replace('</em>', '')
    else:
        return value

def patent_search(q, limit):
  """
  Выполняет поиск по базе данных Роспатент.

  Параметры:
  - q (str): слово или фраза, по которой ищем,
  - limit (int): количество возвращаемых документов.

  Возвращает в случае успеха:
  - (list): содержит информацию об статусе запроса удачный/нет, количество найденных документов, доступных и датафрейм с информацией.
  Возвращает в случае неудачи:
  - (list): содержит информацию об статусе запроса удачный/нет, ошибку, пустой датафрейм.
  """
  url = 'https://searchplatform.rospatent.gov.ru/patsearch/v0.2/search'
  headers = {
      'Authorization': 'ВАШИ ДАННЫЕ',
      'Content-Type': 'application/json'
      }
  data = {
    'q': q,
    'limit' : limit
  }
  try:
    flag = False
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    flag = True
    ans = response.json()
    ans_pd = pd.DataFrame()
    if ans["total"] != 0 and ans["available"] != 0:
      ans_pd = pd.json_normalize(ans["hits"])
      column = ['id', 'common.document_number', 'snippet.title', 'biblio.ru.inventor', 'snippet.applicant', 'snippet.patentee', 'biblio.ru.patentee']
      ans_pd = ans_pd.reindex(columns=column)
      # Применение функции к указанным столбцам
      columns_to_apply = ['biblio.ru.inventor', 'biblio.ru.patentee']
      ans_pd[columns_to_apply] = ans_pd[columns_to_apply].applymap(unpack_names)
      ans_pd = ans_pd.applymap(remove_em_tags)
    return flag, ans["total"], ans["available"], ans_pd
  except requests.exceptions.HTTPError as err:
    return flag, err, pd.DataFrame()
  except requests.exceptions.RequestException as err:
    return flag, err, pd.DataFrame()

In [ ]:
def info_patent(key_word, multi_key, often_word):
  """
  Выполняет поиск по базе данных Роспатент.

  Параметры:
  - key_word (list): ключевые слова,
  - multi_key (list): ключевые фразы,
  - often_word (list): часто встречающиеся слова.

  Возвращает:
  - dict_patent (dict): содержит информацию о выполненных поисках в базе.
  """
  dict_patent = {}
  limit = 5
  dict_patent["all_keywords"] = patent_search(" AND ".join(key_word), limit)
  if len(key_word) >= 5:
    dict_patent["five_keywords"] = patent_search(" AND ".join(often_word[:5]), limit)
  for i in key_word:
    dict_patent[i] = patent_search(i, 1)
  dict_patent["or_multi"] = patent_search(" OR ".join(key_word), limit)
  return dict_patent

###OpenAlex

In [ ]:
def openalex(key, num=1):
  """
  Выполняет поиск по базе данных OpenAlex.

  Параметры:
  - key (list): ключевые слова,
  - num (int): количество возвращаемых значений по ключевому слову.

  Возвращает:
  - result_df (DataFrame): содержит информацию о выполненных поисках в базе.
  """
  columns = ["Слово", "doi",  "Заглавие", 'Индексируется в', 'Наличие полного текста', 'api url']
  url = f"https://api.openalex.org/works?sample={num}"
  result_df = pd.DataFrame(columns=columns)
  # Параметры запроса
  for i in key:
    params = {"filter": f"title.search:{i}"}
    # Выполнение запроса
    response = requests.get(url, params=params)
    # Проверка статуса ответа
    if response.status_code == 200:
      data = response.json()
      df_open = pd.DataFrame.from_dict(data["results"])
      df_open = df_open.reindex(columns=["doi",  "title", 'indexed_in', 'has_fulltext', 'cited_by_api_url'])
      df_open = df_open[["doi",  "title", 'indexed_in', 'has_fulltext', 'cited_by_api_url']]
      df_open['indexed_in'] = df_open['indexed_in'].apply(lambda x: ", ".join(x))
      if len(df_open) != 0:
        new_row = pd.DataFrame([[i] + list(df_open.iloc[0])], columns=result_df.columns)
        result_df = pd.concat([new_row, result_df], ignore_index=True)
        result_df = result_df.replace(False, "Нет").fillna("")
    else:
      print(f"Ошибка при выполнении запроса: для слова {i}", response.status_code)
  return result_df

### Поиск именнованных сущностей

In [ ]:
#!pip install natasha

In [ ]:
import natasha
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, LOC, ORG, NamesExtractor, AddrExtractor, Doc

def name_org_loc(Type_poisk, segmenter, morph_tagger, syntax_parser, ner_tagger, morph_vocab, extractor, text):
  """
  Выполняет поиск именнованных сущностей.

  Параметры:
  - Type_poisk (str): тип данных, который ищем,
  - segmenter (natasha.segment.Segmenter): класс для сегментирования текста на токены,
  - morph_tagger (natasha.morph.tagger.NewsMorphTagger): морфологический теггер,
  - syntax_parser (natasha.syntax.NewsSyntaxParser): синтаксический парсер,
  - ner_tagger (natasha.morph.vocab.MorphVocab): словарный запас морфологических единиц,
  - extractor (natasha.extractors.AddrExtractor): словарный запас адресов,
  - text (str): текст.

  Возвращает:
  - (list): найденные данные.
  """
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)
  for span in doc.spans:
    span.normalize(morph_vocab)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  for span in doc.spans:
    if span.type == Type_poisk:
      span.extract_fact(extractor)
  res_dict = {_.normal: _.fact.as_dict for _ in doc.spans if _.fact}
  return list(res_dict.keys())

In [ ]:
def entities(text):
  """
  Выполняет поиск именнованных сущностей PER, LOC, ORG.

  Параметры:
  - text (str): текст.

  Возвращает:
  - (list): найденные данные.
  """
  segmenter = Segmenter()
  morph_vocab = MorphVocab()
  emb = NewsEmbedding()
  morph_tagger = NewsMorphTagger(emb)
  syntax_parser = NewsSyntaxParser(emb)
  ner_tagger = NewsNERTagger(emb)
  names_extractor = NamesExtractor(morph_vocab)
  addr_extractor = AddrExtractor(morph_vocab)
  res = []
  for item1, item2 in zip([PER, LOC, ORG], [names_extractor, addr_extractor, names_extractor]):
    res.append(name_org_loc(item1, segmenter, morph_tagger, syntax_parser, ner_tagger, morph_vocab, item2, text))
  return res

### ФОРМИРОВАНИЕ ОТЧЕТА ИЗ ДАННЫХ

In [ ]:
#!pip install python-docx
#!pip install aspose-words

In [ ]:
def table_docx(answer, doc):
    """
    Функция добавления таблицы в отчет.
    """
    table = doc.add_table(1, len(answer.columns))
    table.style = 'Light Grid Accent 1'
    head_cells = table.rows[0].cells
    for i, item in enumerate(answer.columns):
        p = head_cells[i].paragraphs[0]
        p.add_run(item).bold = True
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    for _, row in answer.iterrows():
        cells = table.add_row().cells
        for i, item in enumerate(row):
            cells[i].text = str(item)

In [ ]:
def table_patent(dict_patent, paragraph, doc):
  """
  Функция добавления данных из Роспатента.
  """
  if dict_patent[0] == False:
    run = paragraph.add_run(f"ОШИБКА: {dict_patent[0]}")
  else:
    run = paragraph.add_run(f"Количество найденных документов: {dict_patent[1]}\n")
    run = paragraph.add_run(f"Количество доступных документов: {dict_patent[2]}")
    if dict_patent[1] != 0 and dict_patent[2] != 0:
      data = dict_patent[3]
      data = data.replace("nan", "").fillna("")
      data.columns = ["id", "Номер документа", "Название", "Изобретатель", "Заявитель", "Патентообладатель" ,"Патентообладатели"]
      table = doc.add_table(1, len(data.columns))
      table.style = 'Light Grid Accent 1'
      head_cells = table.rows[0].cells
      for i, item in enumerate(data.columns):
        p = head_cells[i].paragraphs[0]
        p.add_run(item).bold = True
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
      for _, row in data.iterrows():
        cells = table.add_row().cells
        for i, item in enumerate(row):
          cells[i].text = str(item)

In [ ]:
def par_docx(doc, dict_patent):
  """
  Функция добавления данных из Роспатента.
  """
  key_info = []
  for key, value in dict_patent.items():
    if key == "all_keywords":
      paragraph = doc.add_paragraph()
      run = paragraph.add_run("Информация по патентам в которых встречается каждое ключевое слово:\n")
      run.font.color.rgb = RGBColor(0, 240, 0)
      table_patent(value, paragraph, doc)
    elif key == "five_keywords":
      paragraph1 = doc.add_paragraph()
      run1 = paragraph1.add_run("Информация по патентам в которых встречается 5 самых частовстречающися слов:\n")
      run1.font.color.rgb = RGBColor(0, 240, 0)
      table_patent(value, paragraph1, doc)
    elif key == "or_multi":
      paragraph2 = doc.add_paragraph()
      run2 = paragraph2.add_run("Информация по патентам в которых встречается одна из ключевых фраз:\n")
      run2.font.color.rgb = RGBColor(0, 240, 0)
      table_patent(value, paragraph2, doc)
    else:
      if value[0] != False and (value[1] != 0 and value[2] != 0):
        key_info.append([key, value[1], value[2], value[3]])
      else:
        print(key, value[0], value[1], value[2])
  columns = ['Ключевое слово', 'Найденные документы', 'Доступные документы', "id", "Номер документа", "Название", "Изобретатель", "Заявитель", "Патентообладатель" ,"Патентообладатели"]
  result_df = pd.DataFrame(columns=columns)
  for array in key_info:
    new_row = pd.DataFrame([[array[0], array[1], array[2]] + list(array[3].iloc[0])], columns=result_df.columns)
    result_df = pd.concat([new_row, result_df], ignore_index=True)
  paragraph3 = doc.add_paragraph()
  run3 = paragraph3.add_run("\nИнформация по патентам в которых встречается одно ключевое слово:")
  run3.font.color.rgb = RGBColor(0, 240, 0)
  result_df = result_df.replace("nan", "").fillna("")
  table_docx(result_df, doc)

In [ ]:
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
import aspose.words as aw
from docx.shared import Inches


def report_docx(result_gost, answer, df_often, key_word, multi_df, patent, arr_entitie, data_openalex, patent_flag, data_openalex_flag):
  """
  Функция формирования отчета.
  """
  substrings = ["Дата окончания проверки текста на сервере:", "Уникальность текста:", "Список URL и процент совпадения текста:", "Произошла ошибка"]
  doc = Document()
  style = doc.styles['Normal']
  style.font.name = 'Arial'
  style.font.size = Pt(11)
  head = doc.add_heading('Отчет автоматической проверки текста')
  head.style.font.size = Pt(20)
  head.alignment = WD_ALIGN_PARAGRAPH.CENTER
  head.style.paragraph_format.space_before = Pt(0)
  paragraph = doc.add_paragraph()
  run = paragraph.add_run('Информация о соответствии структуре ГОСТ\n')
  run.font.size = Pt(16)
  run.bold = True
  run.underline = True
  paragraph.space_after = Pt(12)
  for ans in result_gost:
    run_1 = paragraph.add_run(f"{ans[0]}: ")
    if ans[1] == True:
      run_2 = paragraph.add_run(f"Да")
    else:
      run_2 = paragraph.add_run(f"Нет")
    if ans[0] != result_gost[-1][0]:
      paragraph.add_run("\n")
  paragraph = doc.add_paragraph()
  run = paragraph.add_run('Информация из антиплагиата\n')
  run.font.size = Pt(16)
  run.bold = True
  run.underline = True
  paragraph.space_after = Pt(12)
  if "Произошла ошибка" not in answer[0]:
    for ans in answer[0]:
      for st in substrings:
        if st in ans:
          cur_str = ans.replace(st, "")
          run_1 = paragraph.add_run(st)
          run_1.font.bold = True
          if cur_str != "\n":
            paragraph.add_run(cur_str)
          break
    table_docx(answer[1], doc)
  else:
    run_1 = paragraph.add_run(f"{answer[0]}\n")
  paragraph = doc.add_paragraph()
  run = paragraph.add_run('Ключевые слова и фразы из текста\n')
  run.font.size = Pt(16)
  run.bold = True
  run.underline = True
  paragraph.space_after = Pt(12)
  run = paragraph.add_run('Облако слов\n')
  run.add_picture('wordcloud.png', width=Inches(5))
  run = paragraph.add_run("\n10 самых частовстречающихся слов: ")
  run.font.color.rgb = RGBColor(0, 240, 0)
  run.italic = True
  paragraph.add_run(', '.join(list(df_often["word"])[:10]))
  run1 = paragraph.add_run("\nКлючевые слова: ")
  run1.font.color.rgb = RGBColor(0, 240, 0)
  run1.italic = True
  paragraph.add_run(', '.join(key_word))
  run2 = paragraph.add_run("\nКлючевые словосочетания: ")
  run2.font.color.rgb = RGBColor(0, 240, 0)
  run2.italic = True
  paragraph.add_run(', '.join(multi_df["word"]))
  run3 = paragraph.add_run("\nДанные найденные по ключевым словам из Роспатента")
  run3.font.size = Pt(16)
  run3.bold = True
  run3.underline = True
  paragraph.space_after = Pt(12)
  if "Нет ключевых слов и фраз для поиска." == patent_flag:
    paragraph.add_run("\nНет ключевых слов и фраз для поиска.\n")
  else:
    k = par_docx(doc, patent)
  paragraph = doc.add_paragraph()
  run4 = paragraph.add_run('\nЛокации, ФИО и организации из текста\n')
  run4.font.size = Pt(16)
  run4.bold = True
  run4.underline = True
  paragraph.space_after = Pt(12)
  run5 = paragraph.add_run('\nНайденные ФИО: ')
  run5.font.color.rgb = RGBColor(0, 240, 0)
  run5.italic = True
  paragraph.add_run(', '.join(arr_entitie[0]))
  run6 = paragraph.add_run('\nНайденные локации: ')
  run6.font.color.rgb = RGBColor(0, 240, 0)
  run6.italic = True
  paragraph.add_run(', '.join(arr_entitie[1]))
  run7 = paragraph.add_run('\nНайденные организации: ')
  run7.font.color.rgb = RGBColor(0, 240, 0)
  run7.italic = True
  paragraph.add_run(', '.join(arr_entitie[2]))
  paragraph = doc.add_paragraph()
  run8 = paragraph.add_run('Данные найденные по ключевым словам из OpenAlex')
  run8.font.size = Pt(16)
  run8.bold = True
  run8.underline = True
  paragraph.space_after = Pt(12)
  if "Нет ключевых слов и фраз для поиска." == data_openalex_flag:
    paragraph.add_run("\nНет ключевых слов и фраз для поиска.\n")
  else:
    table_docx(data_openalex, doc)
  doc.save('Отчет.docx')

### Формирование отчета

In [ ]:
import time
import plotly.express as px
import plotly.io as pio

def automatic_report(link, document_type):
  """
  Формирует и анализирует предоставленный текст.

  Параметры:
  - link (str): ссылка на файл,
  - document_type (int): тип документа.
  """
  start_time1 = time.time()
  text = extract_text(link)
  end_time1 = time.time()
  if text != "Формат файла не соответствует!" and document_type in [0, 1, 2, 3]:
    #проверка документа на соответствие структуре ГОСТ
    start_time2 = time.time()
    result_gost = gost(text, document_type)
    end_time2 = time.time()
    # Учетные данные с сайта
    userkey = 'ВАШ КЛЮЧ'
    #очистка текста
    cleaned_text = clean_text(text)
    #отправка на проверку в антиплагиате
    start_time3 = time.time()
    if len(cleaned_text) > 150000:
      answer = check_anti_plagiarism(userkey, cleaned_text[:1500]) #максимальное возможное количество для проверки в данном плагиате
    else:
      answer = check_anti_plagiarism(userkey, cleaned_text)
    end_time3 = time.time()
    if "Произошла ошибка" not in answer[0]:
      numb = (answer[1]["%"].sum() / 100) / ((100 - float(answer[2])) / 100)
      df_plagiat = answer[1]
      df_plagiat["%"] = answer[1]["%"].apply(lambda x: x / numb)
      labels = df_plagiat['URL'].tolist() + ['Уникальный текст']
      sizes = df_plagiat['%'].tolist() + [float(answer[2])]
      # Создание кольцевой диаграммы с выделением одного сегмента
      fig = px.pie(names=labels, values=sizes, hole=0.4)
      fig.update_traces(pull=[0.1 if label == 'Уникальный текст' else 0 for label in labels], textposition='inside', insidetextorientation='radial')
      fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=1.1, xanchor='left', x=0.1))
      fig.show()
    # Выделение ключевых слов и часто встречающихся из текста
    start_time4_1 = time.time()
    df_often = pd.DataFrame(sorted(Text_processing(cleaned_text).items(), key=lambda x: x[1], reverse=True), columns=['word', 'count'])
    end_time4_1 = time.time()
    # Орисовка облака слов
    word_cloud(df_often["word"])
    # RAKE
    stops_word = ["менее", "более", "кроме", "должно", "должный", "данный", "детея", "которая", "который"]
    stops = list(set(stopwords.words("russian") + stops_word))
    start_time4_2 = time.time()
    words1 = key_word_rake(text, stops, 5, 1)
    end_time4_2 = time.time()
    key_word1 = [i[0] for i in words1]
    # YAKE
    start_time4_3 = time.time()
    words2 = key_word_yake(cleaned_text, 1, 5)
    end_time4_3 = time.time()
    key_word2 = [i[0].lower() for i in words2]
    # TextRank
    start_time4_4 = time.time()
    words3 = key_word_textrank(cleaned_text, stops, 5)
    end_time4_4 = time.time()
    key_word3 = [i for i in words3]
    # Topia
    start_time4_5 = time.time()
    key_word4, single_word_terms, multi_word_terms = topia(cleaned_text, 5)
    end_time4_5 = time.time()
    start_time4 = start_time4_1 + start_time4_2 + start_time4_3 + start_time4_4 + start_time4_5
    end_time4 = end_time4_1 + end_time4_2 + end_time4_3 + end_time4_4 + end_time4_5
    # Ключевые фразы
    multi_df = pd.DataFrame(multi_word_terms[:15], columns=['word', 'count'])
    # Лемматизация слов в каждом массиве
    lemmatized_array1 = lemmatize_words(key_word1)
    lemmatized_array2 = lemmatize_words(key_word2)
    lemmatized_array3 = lemmatize_words(key_word3)
    lemmatized_array4 = lemmatize_words(key_word4)
    lemmatized_array5 = lemmatize_words(df_often['word'].iloc[:5].values)
    # Ключевые слова + часто встречающиеся
    res_word = lemmatized_array1 | lemmatized_array2 | lemmatized_array3 | lemmatized_array4 |lemmatized_array5
    # Ключевые слова
    key_word = lemmatized_array1 | lemmatized_array2 | lemmatized_array3 | lemmatized_array4
    start_time5, start_time7 = 0, 0
    end_time5, end_time7 = 0, 0
    patent_flag, data_openalex_flag = True, True
    if len(list(res_word)) != 0 or len(multi_df["word"]) != 0:
      # Поиск по Роспатенту
      start_time5 = time.time()
      patent = info_patent(list(res_word), multi_df["word"], df_often['word'])
      end_time5 = time.time()
      len1 = 3 + len(list(res_word))
      # Поиск по OpenAlex
      start_time7 = time.time()
      data_openalex = openalex(list(res_word) + list(multi_df["word"].values))
      end_time7 = time.time()
      len2 = len(list(res_word) + list(multi_df["word"].values))
    else:
      patent_flag = False
      data_openalex_flag = False
    # Поиск именнованных сущностей
    start_time6 = time.time()
    arr_entitie = entities(cleaned_text)
    end_time6 = time.time()
    # Формирование отчета
    start_time8 = time.time()
    report_docx(result_gost, answer, df_often, key_word, multi_df, patent, arr_entitie, data_openalex, patent_flag, data_openalex_flag)
    end_time8 = time.time()
    dict_type_doc = {0: "Техническое задание",
                     1: "Пояснительная записка",
                     2: "Отчет НИР",
                     3: "Отчет об опытной эксплуатации"}
    return link.split('/')[-1], dict_type_doc[document_type], len(text), len(cleaned_text), end_time1 - start_time1, end_time2 - start_time2, end_time3 - start_time3, end_time4 - start_time4, end_time5 - start_time5, end_time6 - start_time6, end_time7 - start_time7, end_time8 - start_time8, end_time4_1 - start_time4_1, end_time4_2 - start_time4_2, end_time4_3 - start_time4_3, end_time4_4 - start_time4_4, end_time4_5 - start_time4_5, len1, len2
  else:
    print("Невозможно создать отчет, проверьте подаваемый документ!")
    return "ОШИБКА"

### Анализ

In [ ]:
arr_link = ["Документ 1",
            "Документ 2",
            "Документ 3",
            "Документ 4",
            "Документ 5",
            "Документ 6",
            "Документ 7",
            "Документ 8",
            "Документ 9",
            "Документ 10",
            "Документ 11",
            "Документ 12"]
arr_type = [0,0,0,0,0,1,1,2,2,3,3,3]

In [ ]:
dict_type_doc = {0: "Техническое задание",
                     1: "Пояснительная записка",
                     2: "Отчет НИР",
                     3: "Отчет об опытной эксплуатации"}

In [ ]:
result1 = pd.DataFrame({'Документ': [], 'Тип документа': [], 'Количество символов': [], 'Количество символов после очистки': [], 'Время перевода документа в машинночитаемый вид': [], 'Время проверки на соответствие структуре ГОСТ': [], 'Время проверки документа в системе антиплагиат': [], 'Общее время выделения ключевых и часто встречающихся слов и фраз': [], 'Время поиска по Роспатенту': [], 'Время поиска именнованных сущностей': [], 'Время поиска по OpenAlex': [], 'Время формирования отчета': [], "Количество получившихся запросов к Роспатенту": [], "Количество получившихся запросов к OpenAlex": []})
result2 = pd.DataFrame({'Документ': [], 'Тип документа': [], 'Общее время выделения часто встречающихся слов и фраз': [], 'Время выделения часто встречающихся слов': [], 'Время выделения ключевых слов методом RAKE': [], 'Время выделения ключевых слов методом YAKE': [], 'Время выделения ключевых слов методом TextRank': [], 'Времы выделения ключевых слов методом Topia': []})

In [ ]:
for i, j in zip(arr_link, arr_type):
  res = automatic_report(i, j)
  if res != "ОШИБКА":
    result1.loc[len(result1.index)] = [res[0], res[1], res[2], res[3], res[4],  res[5], res[6], res[7], res[8], res[9], res[10], res[11], res[17], res[18]]
    result2.loc[len(result2.index)] = [res[0], res[1], res[7], res[12],  res[13],  res[14],  res[15],  res[16]]
    result1.to_excel("адрес сохранения")
    result2.to_excel("адрес сохранения")
  else:
    print(f"Ошибка при обработки документа {link}")

In [ ]:
import pandas as pd

result1 = pd.read_excel("адрес сохранения")
result2 = pd.read_excel("адрес сохранения")
result1 = result1.drop('Unnamed: 0', axis=1)
result2 = result2.drop('Unnamed: 0', axis=1)

In [ ]:
result1.iloc[:, 2:].mean()

Количество символов                                                 178317.916667
Количество символов после очистки                                   163990.083333
Время перевода документа в машинночитаемый вид                           1.285627
Время проверки на соответствие структуре ГОСТ                            0.073782
Время проверки документа в системе антиплагиат                          84.719000
Общее время выделения ключевых и часто встречающихся слов и фраз        46.842275
Время поиска по Роспатенту                                              98.995493
Время поиска именнованных сущностей                                     43.933758
Время поиска по OpenAlex                                                12.603817
Время формирования отчета                                                0.415532
Количество получившихся запросов к Роспатенту                           18.750000
Количество получившихся запросов к OpenAlex                             29.916667
dtype: float64

In [ ]:
result2.iloc[:, 2:].mean()

Общее время выделения часто встречающихся слов и фраз    46.842275
Время выделения часто встречающихся слов                  4.522322
Время выделения ключевых слов методом RAKE                0.301547
Время выделения ключевых слов методом YAKE                1.873420
Время выделения ключевых слов методом TextRank           31.867459
Времы выделения ключевых слов методом Topia               8.277528
dtype: float64

In [ ]:
columns_to_sum = result1.columns[4:]
row_sums = result1[columns_to_sum].sum(axis=1)
print("Суммы для каждой строки:")
print(row_sums)
print(f"Среднее значение всех сумм: {row_sums.mean()}")

Суммы для каждой строки:
0     233.694684
1     329.859509
2     304.619835
3     341.871475
4     295.120563
5     216.840783
6     320.051265
7     416.430831
8     363.665913
9     505.750322
10    376.946660
11    345.579582
dtype: float64
Среднее значение всех сумм: 337.53595185415975


In [ ]:
for group_name, group_data in result1.groupby(result1[result1.columns[1]]):
    row_sums = group_data[result1.columns[4:]].sum(axis=1)
    print(f"Группа: {group_name}")
    print("Суммы для каждой строки:")
    print(row_sums)
    print(f"Среднее значение всех сумм: {row_sums.mean()}")

Группа: Отчет НИР
Суммы для каждой строки:
9     505.750322
10    376.946660
dtype: float64
Среднее значение всех сумм: 441.3484911311492
Группа: Отчет об опытной эксплуатации
Суммы для каждой строки:
4     295.120563
5     216.840783
11    345.579582
dtype: float64
Среднее значение всех сумм: 285.84697600682347
Группа: Пояснительная записка
Суммы для каждой строки:
0    233.694684
2    304.619835
dtype: float64
Среднее значение всех сумм: 269.15725936889646
Группа: Техническое задание
Суммы для каждой строки:
1    329.859509
3    341.871475
6    320.051265
7    416.430831
8    363.665913
dtype: float64
Среднее значение всех сумм: 354.375798645871


In [ ]:
337.53595185415975 / 60

5.625599197569329

In [ ]:
result2.iloc[:, 2:].mean()

Общее время выделения часто встречающихся слов и фраз    46.842275
Время выделения часто встречающихся слов                  4.522322
Время выделения ключевых слов методом RAKE                0.301547
Время выделения ключевых слов методом YAKE                1.873420
Время выделения ключевых слов методом TextRank           31.867459
Времы выделения ключевых слов методом Topia               8.277528
dtype: float64

In [ ]:
for group_name, group_data in result2.groupby(result2[result2.columns[1]]):
    row_sums = group_data[result2.columns[3:]].sum(axis=1)
    print(f"Группа: {group_name}")
    print("Суммы для каждой строки:")
    print(row_sums)
    print(f"Среднее значение всех сумм: {row_sums.mean()}")

Группа: Отчет НИР
Суммы для каждой строки:
9     185.592427
10     48.993207
dtype: float64
Среднее значение всех сумм: 117.29281675815524
Группа: Отчет об опытной эксплуатации
Суммы для каждой строки:
4     21.317083
5      6.522424
11    90.491787
dtype: float64
Среднее значение всех сумм: 39.44376452763872
Группа: Пояснительная записка
Суммы для каждой строки:
0     7.879074
2    31.415858
dtype: float64
Среднее значение всех сумм: 19.64746594429016
Группа: Техническое задание
Суммы для каждой строки:
1    23.326828
3    25.873461
6    28.336969
7    65.190987
8    27.167203
dtype: float64
Среднее значение всех сумм: 33.979089450836184


In [ ]:
columns_to_round = result1.columns[2:]
result1[columns_to_round] = result1[columns_to_round].round(decimals=1)
file_path = "адрес сохранения"
result1.to_excel(file_path, index=False)

In [ ]:
columns_to_round = result2.columns[2:]
result2[columns_to_round] = result2[columns_to_round].round(decimals=1)
file_path = "адрес сохранения"
result2.to_excel(file_path, index=False)